In [1]:
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel

import pynndescent
import numpy as np
import sys
import os
from tqdm import tqdm
import sqlite3
sys.path.insert(0, '..')
sys.path.insert(0, '../data_processing')
sys.path.insert(0, '../datasets')
import my_utils
from ComponentAutoExtractor import ComponentAutoExtractor


/home/grads/t/tiendat.ng.cs/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# This experiment assert the how well sbert perform given only the short desc of BR

## Preparation

In [2]:
home_path = "/home/grads/t/tiendat.ng.cs/github_repos/PLM_and_BugReport_datasets"
data_path = os.path.join(home_path, "datasets", "hand-gen-datasets")

# connect to db
database_path = os.path.join(home_path, "dbrd_processed.db")
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

In [3]:
# process db table, create save folder
table = "spark"
save_path = os.path.join(data_path, table)
my_utils.create_folder(save_path)

union_find = my_utils.UnionFind()
union_find.process_project(conn, table, min_desc_length=10)
bug_ids = my_utils.get_bug_ids(conn, table)
bug_ids_w_duplicates = union_find.get_all_children()

# loop through each desc extract components, and save to file
print("Number of bug_ids before filter: ", len(bug_ids))
# remove bug_ids that are of very short desc and those that does not have log
to_remove_ids = []
for bug_id in bug_ids:
    desc = my_utils.get_description(conn, table, bug_id)
    # short_desc = my_utils.get_short_desc(conn, table, bug_id)
    auto_extractor = ComponentAutoExtractor(desc)
    if len(desc) < 50 or not auto_extractor.has_log() or bug_id not in bug_ids_w_duplicates:
        to_remove_ids.append(bug_id)

for to_remove_id in to_remove_ids:
    bug_ids.remove(to_remove_id)

print("Number of bug_ids after filter: ", len(bug_ids))

Folder '/home/grads/t/tiendat.ng.cs/github_repos/PLM_and_BugReport_datasets/datasets/hand-gen-datasets/spark' already exists.
Processing spark


  0%|          | 0/9579 [00:00<?, ?it/s]

100%|██████████| 9579/9579 [00:00<00:00, 84177.66it/s]


Number of bug_ids before filter:  9577
Number of bug_ids after filter:  124


## Load model

In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [5]:
# finalize search space by adding duplicates of bug_ids_w_duplicates_and_stacktrace
search_space = bug_ids.copy()
for bug_id in tqdm(bug_ids):
    dups = union_find.get_children(bug_id)
    for dup in dups:
        if dup != bug_id and dup not in search_space:
            search_space.append(dup)

100%|██████████| 124/124 [00:00<00:00, 46070.84it/s]


In [6]:
# convert to vectors

search_space_vects = {}
for bug_id in tqdm(search_space):
    desc = my_utils.get_description(conn, table, bug_id)
    vect = model.encode(desc,convert_to_tensor=True).numpy()
    search_space_vects[bug_id] = vect

100%|██████████| 175/175 [00:05<00:00, 33.54it/s]


In [7]:
index = pynndescent.NNDescent(np.array(list(search_space_vects.values())), n_neighbors=50, metric="cosine")
index.prepare()

In [8]:

Q_vects = []
Q_indices = [search_space.index(bug_id) for bug_id in bug_ids]
for bug_id in tqdm(bug_ids):
    # eng = segregate_log_and_stacktrace(my_utils.get_descriptions(conn, table, bug_id))[0]
    vect = search_space_vects[bug_id]
    Q_vects.append(vect)

100%|██████████| 124/124 [00:00<00:00, 874107.05it/s]


In [9]:
neighbors = index.query(np.array(Q_vects), 11)

In [15]:
found_in_top_k_wo_stacktrace = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
for i in tqdm(range(len(bug_ids))):
    q = bug_ids[i]
    # print("Q = ", q)
    # print("Index of query ", Q_indices[i])
    # print("Index of neighbors ", neighbors[0][i])
    # print("Duplicates ", union_find.get_children(q))
    index_of_duplicates = [search_space.index(id) for id in union_find.get_children(q)]
    # print("Index of duplicates ", index_of_duplicates)
    for result_k in range(1, len(neighbors[0][i][:])):
        if neighbors[0][i][result_k] in index_of_duplicates:
            # increment from k to 10
            for f in range(result_k, len(found_in_top_k_wo_stacktrace)):
                found_in_top_k_wo_stacktrace[f] += 1
            break
        

100%|██████████| 124/124 [00:00<00:00, 30173.10it/s]


In [16]:
found_in_top_k_wo_stacktrace / len(bug_ids)

array([0.        , 0.69354839, 0.71774194, 0.72580645, 0.75806452,
       0.75806452, 0.77419355, 0.7983871 , 0.80645161, 0.80645161,
       0.82258065])

In [26]:
i = 9
q = bug_ids[i]
index_of_duplicates = [search_space.index(id) for id in union_find.get_children(q)]
print(index_of_duplicates)
print(neighbors[0][i])

[12, 9, 129]
[  9 129  12  94  21  18 126  86  38  44  67]


## Pool is whole table